<a href="https://colab.research.google.com/github/bkoyuncu/courses/blob/master/Basketball_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Kaynak \\
Programs for "Numerical Methods for Physics" by Alejandro Garcia \\
Alejandro Garcia Dept. Physics and Astronomy San Jose State University  Alejandro.Garcia@sjsu.edu \\
https://github.com/AlejGarcia/NM4P \\
edit by batu

In [0]:
# edit by batu 12.10.2019
# Basketball - Program to compute the trajectory of a basketball
#         using the Euler method.

# Set up configuration options and special features
%matplotlib inline
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.animation as animation

In [13]:
#* Set initial position and velocity of the basketball
y0 = int(input('Enter initial height (meters): ')   )
r0 = np.array([0, y0])      # Initial vector position
speed = int(input('Enter initial speed (m/s): '))
theta = int(input('Enter initial angle (degrees): ') )
v0 = np.array([speed * np.cos(theta*np.pi/180), speed * np.sin(theta*np.pi/180)])      # Initial velocity
r = np.copy(r0)   # Set initial position 
v = np.copy(v0)   # Set initial velocity

#* Set physical parameters (mass, Cd, etc.)
Cd = 0.54      # Drag coefficient for the basketball in air (dimensionless)
area = 0.18  # Cross-sectional area of projectile (m^2)
grav = 9.81    # Gravitational acceleration (m/s^2)
mass = 0.625   # Mass of projectile (kg)
airFlag = int(input('Air resistance? (Yes:1, No:0): '))
if airFlag == 0 :
    rho = 0.      # No air resistance
else:
    rho = 1.2     # Density of air (kg/m^3)
air_const = -0.5*Cd*rho*area/mass   # Air resistance constant

Enter initial height (meters): 10
Enter initial speed (m/s): 10
Enter initial angle (degrees): 45
Air resistance? (Yes:1, No:0): 1


In [14]:
#* Loop until ball hits ground or max steps completed
tau = float(input('Enter timestep, tau (sec): '))   # (sec)
maxstep = 1000    # Maximum number of steps
xplot = np.empty(maxstep);  yplot = np.empty(maxstep)
xNoAir = np.empty(maxstep); yNoAir = np.empty(maxstep)
k=0
for istep in range(maxstep):
    k+=1
    #* Record position (computed and theoretical) for plotting
    xplot[istep] = r[0]   # Record trajectory for plot
    yplot[istep] = r[1]
    t = istep*tau         # Current time
    xNoAir[istep] = r0[0] + v0[0]*t
    yNoAir[istep] = r0[1] + v0[1]*t - 0.5*grav*t**2
  
    #* Calculate the acceleration of the ball 
    accel = air_const * np.linalg.norm(v) * v   # Air resistance
    accel[1] = accel[1] - grav                  # Gravity
  
    #* Calculate the new position and velocity using Euler method
    r = r + tau*v                    # Euler step
    v = v + tau*accel     
  
    #* If ball reaches ground (y<0), break out of the loop
    if r[1] < 0 : 
        laststep = istep+1
        xplot[laststep] = r[0]  # Record last values computed
        yplot[laststep] = r[1]
        t = laststep*tau         # Current time
        xNoAir[laststep] = r0[0] + v0[0]*t
        yNoAir[laststep] = r0[1] + v0[1]*t - 0.5*grav*t**2
        xplot,yplot = xplot[:laststep+1],yplot[:laststep+1]
        xNoAir,yNoAir = xNoAir[:laststep+1],yNoAir[:laststep+1]
        #yplot[-1]=0
        #yNoAir[-1]=0
        break                   # Break out of the for loop

Enter timestep, tau (sec): 0.1


In [18]:
fig = plt.figure()
ax1 = plt.axes(xlim=(0, 50), ylim=(0,50))
line, = ax1.plot([], [], lw=2)

plotlays, plotcols = [2], ["black","red"]
lines = []
legends = ['Euler Method', 'Theory']
for index in range(2):
    lobj = ax1.plot([],[],lw=2,color=plotcols[index],label=legends[index])[0]
    lines.append(lobj)

#print(lines)

x1,y1 = [],[]
x2,y2 = [],[]


def animate(i):


    xlist = [xplot[0:i], xNoAir[:i]]
    ylist = [yplot[0:i], yNoAir[:i]]
    #print(xlist,ylist)
    #for index in range(0,1):
    for lnum,line in enumerate(lines):
        line.set_data(xlist[lnum], ylist[lnum]) # set data for each line separately. 

    return lines

# call the animator.  blit=True means only re-draw the parts that have changed.
anim = animation.FuncAnimation(fig, animate,
                               frames=len(xplot), interval=100)
plt.legend(loc='upper right')
plt.close()
from IPython.display import HTML
HTML(anim.to_jshtml())
